### 1 Readying Clauses

#### Importing Clauses

In [3]:
import ast #helps to change string into list below

In [4]:
f = open("clauses.txt","r")
clauses=ast.literal_eval(f.read())

#### Scrambling Clauses via random.sample

In [6]:
import random #used to shuffle lists

In [7]:
eight_lists_of_clauses=[]
for i in range(8):
    eight_lists_of_clauses.append(random.sample(clauses,len(clauses)))

Running some simple checks

In [9]:
print(len(eight_lists_of_clauses))
print(len(eight_lists_of_clauses[1]))
print(eight_lists_of_clauses[1][:10])

8
144873
[[127, -18916], [10957, -11425, -11833, -31445], [245, -13622], [23, -3636], [18468, 18868, 19248, -22926], [-72, -270, 10854], [22723, 22724, -22725, 22734], [105, -15722], [151, -20437], [3975, -4608, -4786, -25920]]


### 2 starting SAT

In the following line I import a solve. Note that at the moment Minisat22 is chosen as Solver, but that this can be changed without changing anything else in the document - syntax seems to be consistent across solvers

In [12]:
from pysat.solvers import Minisat22 as Solver

#### Load function

It seems silly, but there were some difficulties automating the following

In [15]:
s1=Solver()
s2=Solver()
s3=Solver()
s4=Solver()
s5=Solver()
s6=Solver()
s7=Solver()
s8=Solver()

In [16]:
names=[s1,s2,s3,s4,s5,s6,s7,s8]
for i in range(8):
    names[i].append_formula(eight_lists_of_clauses[i])

Now the solvers are loaded with a different shuffle of the same 144k clauses

### 3 Multiprocessing

In [19]:
from multiprocess import Event, Process

In [20]:
import time

In [21]:
def worker(i,name,event):
    print(f"Worker s{i+1}: I've got the problem! Working hard on it...")
    result=name.solve()
    time.sleep(1)
    print(f's{i+1} yields {result}')
    event.set()  # Trigger the event

In [22]:
def main():
    t0=time.time() #saving initial time
    
    start_event = Event() # Create the event
    
    # Create and start the worker processes
    processes=[]
    for i in range(8):
        p=Process(target=worker, args=(i,names[i],start_event))
        processes.append(p)
        
    print('Main: setting the workers to work')
    for p in processes:
        p.start()

    print("Main: Waiting for a worker to finish.")
    start_event.wait()  # Main process will wait here until event is set
    print("Main: One worker has finished")

    print("Main: Stopping other workers")
    
    for p in processes:
        p.terminate()

    time_spent=(time.time()-t0)/(60*60)
    print(f"Main: Process is finished after {time_spent} hours")

In [23]:
if __name__ == '__main__':
    main()

Main: setting the workers to work
Worker s1: I've got the problem! Working hard on it...
Worker s2: I've got the problem! Working hard on it...
Worker s3: I've got the problem! Working hard on it...
Worker s4: I've got the problem! Working hard on it...
Worker s5: I've got the problem! Working hard on it...
Worker s6: I've got the problem! Working hard on it...
Worker s7: I've got the problem! Working hard on it...
Main: Waiting for a worker to finish.
Worker s8: I've got the problem! Working hard on it...


Process Process-6:
Process Process-3:
Process Process-2:
Process Process-8:
Process Process-5:
Process Process-4:
Process Process-7:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/site-packages/multiprocess/process.py"

KeyboardInterrupt: 